## Test with Regression Activation Vector

In this notebook we calculate the sensitivity score and the Br score

In [44]:
import os
import time
import shutil
import zipfile
import tarfile
import urllib
import requests
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict, Counter
import h5py as h5
import argparse

In [2]:
import numpy as np
import awkward as ak
import uproot
import vector
vector.register_awkward()

In [3]:
import matplotlib.pyplot as plt
import torch

/Users/billyli/miniforge_x86_new/envs/weaver/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
from torch.utils.data import DataLoader
from weaver.utils.dataset import SimpleIterDataset
from weaver.train import test_load, model_setup
from weaver.utils.logger import _logger, _configLogger

## Mimic Args

Copied from train.py

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--regression-mode', action='store_true', default=False,
                    help='run in regression mode if this flag is set; otherwise run in classification mode')
parser.add_argument('-c', '--data-config', type=str,
                    help='data config YAML file')
parser.add_argument('--extra-selection', type=str, default=None,
                    help='Additional selection requirement, will modify `selection` to `(selection) & (extra)` on-the-fly')
parser.add_argument('--extra-test-selection', type=str, default=None,
                    help='Additional test-time selection requirement, will modify `test_time_selection` to `(test_time_selection) & (extra)` on-the-fly')
parser.add_argument('-i', '--data-train', nargs='*', default=[],
                    help='training files; supported syntax:'
                         ' (a) plain list, `--data-train /path/to/a/* /path/to/b/*`;'
                         ' (b) (named) groups [Recommended], `--data-train a:/path/to/a/* b:/path/to/b/*`,'
                         ' the file splitting (for each dataloader worker) will be performed per group,'
                         ' and then mixed together, to ensure a uniform mixing from all groups for each worker.'
                    )
parser.add_argument('-l', '--data-val', nargs='*', default=[],
                    help='validation files; when not set, will use training files and split by `--train-val-split`')
parser.add_argument('-t', '--data-test', nargs='*', default=[],
                    help='testing files; supported syntax:'
                         ' (a) plain list, `--data-test /path/to/a/* /path/to/b/*`;'
                         ' (b) keyword-based, `--data-test a:/path/to/a/* b:/path/to/b/*`, will produce output_a, output_b;'
                         ' (c) split output per N input files, `--data-test a%%10:/path/to/a/*`, will split per 10 input files')
parser.add_argument('--data-fraction', type=float, default=1,
                    help='fraction of events to load from each file; for training, the events are randomly selected for each epoch')
parser.add_argument('--file-fraction', type=float, default=1,
                    help='fraction of files to load; for training, the files are randomly selected for each epoch')
parser.add_argument('--fetch-by-files', action='store_true', default=False,
                    help='When enabled, will load all events from a small number (set by ``--fetch-step``) of files for each data fetching. '
                         'Otherwise (default), load a small fraction of events from all files each time, which helps reduce variations in the sample composition.')
parser.add_argument('--fetch-step', type=float, default=0.01,
                    help='fraction of events to load each time from every file (when ``--fetch-by-files`` is disabled); '
                         'Or: number of files to load each time (when ``--fetch-by-files`` is enabled). Shuffling & sampling is done within these events, so set a large enough value.')
parser.add_argument('--in-memory', action='store_true', default=False,
                    help='load the whole dataset (and perform the preprocessing) only once and keep it in memory for the entire run')
parser.add_argument('--train-val-split', type=float, default=0.8,
                    help='training/validation split fraction')
parser.add_argument('--no-remake-weights', action='store_true', default=False,
                    help='do not remake weights for sampling (reweighting), use existing ones in the previous auto-generated data config YAML file')
parser.add_argument('--demo', action='store_true', default=False,
                    help='quickly test the setup by running over only a small number of events')
parser.add_argument('--lr-finder', type=str, default=None,
                    help='run learning rate finder instead of the actual training; format: ``start_lr, end_lr, num_iters``')
parser.add_argument('--tensorboard', type=str, default=None,
                    help='create a tensorboard summary writer with the given comment')
parser.add_argument('--tensorboard-custom-fn', type=str, default=None,
                    help='the path of the python script containing a user-specified function `get_tensorboard_custom_fn`, '
                         'to display custom information per mini-batch or per epoch, during the training, validation or test.')
parser.add_argument('-n', '--network-config', type=str,
                    help='network architecture configuration file; the path must be relative to the current dir')
parser.add_argument('-o', '--network-option', nargs=2, action='append', default=[],
                    help='options to pass to the model class constructor, e.g., `--network-option use_counts False`')
parser.add_argument('-m', '--model-prefix', type=str, default='models/{auto}/network',
                    help='path to save or load the model; for training, this will be used as a prefix, so model snapshots '
                         'will saved to `{model_prefix}_epoch-%%d_state.pt` after each epoch, and the one with the best '
                         'validation metric to `{model_prefix}_best_epoch_state.pt`; for testing, this should be the full path '
                         'including the suffix, otherwise the one with the best validation metric will be used; '
                         'for training, `{auto}` can be used as part of the path to auto-generate a name, '
                         'based on the timestamp and network configuration')
parser.add_argument('--load-model-weights', type=str, default=None,
                    help='initialize model with pre-trained weights')
parser.add_argument('--exclude-model-weights', type=str, default=None,
                    help='comma-separated regex to exclude matched weights from being loaded, e.g., `a.fc..+,b.fc..+`')
parser.add_argument('--freeze-model-weights', type=str, default=None,
                    help='comma-separated regex to freeze matched weights from being updated in the training, e.g., `a.fc..+,b.fc..+`')
parser.add_argument('--num-epochs', type=int, default=20,
                    help='number of epochs')
parser.add_argument('--steps-per-epoch', type=int, default=None,
                    help='number of steps (iterations) per epochs; '
                         'if neither of `--steps-per-epoch` or `--samples-per-epoch` is set, each epoch will run over all loaded samples')
parser.add_argument('--steps-per-epoch-val', type=int, default=None,
                    help='number of steps (iterations) per epochs for validation; '
                         'if neither of `--steps-per-epoch-val` or `--samples-per-epoch-val` is set, each epoch will run over all loaded samples')
parser.add_argument('--samples-per-epoch', type=int, default=None,
                    help='number of samples per epochs; '
                         'if neither of `--steps-per-epoch` or `--samples-per-epoch` is set, each epoch will run over all loaded samples')
parser.add_argument('--samples-per-epoch-val', type=int, default=None,
                    help='number of samples per epochs for validation; '
                         'if neither of `--steps-per-epoch-val` or `--samples-per-epoch-val` is set, each epoch will run over all loaded samples')
parser.add_argument('--optimizer', type=str, default='ranger', choices=['adam', 'adamW', 'radam', 'ranger'],  # TODO: add more
                    help='optimizer for the training')
parser.add_argument('--optimizer-option', nargs=2, action='append', default=[],
                    help='options to pass to the optimizer class constructor, e.g., `--optimizer-option weight_decay 1e-4`')
parser.add_argument('--lr-scheduler', type=str, default='flat+decay',
                    choices=['none', 'steps', 'flat+decay', 'flat+linear', 'flat+cos', 'one-cycle'],
                    help='learning rate scheduler')
parser.add_argument('--warmup-steps', type=int, default=0,
                    help='number of warm-up steps, only valid for `flat+linear` and `flat+cos` lr schedulers')
parser.add_argument('--load-epoch', type=int, default=None,
                    help='used to resume interrupted training, load model and optimizer state saved in the `epoch-%%d_state.pt` and `epoch-%%d_optimizer.pt` files')
parser.add_argument('--start-lr', type=float, default=5e-3,
                    help='start learning rate')
parser.add_argument('--batch-size', type=int, default=128,
                    help='batch size')
parser.add_argument('--use-amp', action='store_true', default=False,
                    help='use mixed precision training (fp16)')
parser.add_argument('--gpus', type=str, default='0',
                    help='device for the training/testing; to use CPU, set to empty string (""); to use multiple gpu, set it as a comma separated list, e.g., `1,2,3,4`')
parser.add_argument('--predict-gpus', type=str, default=None,
                    help='device for the testing; to use CPU, set to empty string (""); to use multiple gpu, set it as a comma separated list, e.g., `1,2,3,4`; if not set, use the same as `--gpus`')
parser.add_argument('--num-workers', type=int, default=1,
                    help='number of threads to load the dataset; memory consumption and disk access load increases (~linearly) with this numbers')
parser.add_argument('--predict', action='store_true', default=False,
                    help='run prediction instead of training')
parser.add_argument('--predict-output', type=str,
                    help='path to save the prediction output, support `.root` and `.parquet` format')
parser.add_argument('--export-onnx', type=str, default=None,
                    help='export the PyTorch model to ONNX model and save it at the given path (path must ends w/ .onnx); '
                         'needs to set `--data-config`, `--network-config`, and `--model-prefix` (requires the full model path)')
parser.add_argument('--onnx-opset', type=int, default=15,
                    help='ONNX opset version.')
parser.add_argument('--io-test', action='store_true', default=False,
                    help='test throughput of the dataloader')
parser.add_argument('--copy-inputs', action='store_true', default=False,
                    help='copy input files to the current dir (can help to speed up dataloading when running over remote files, e.g., from EOS)')
parser.add_argument('--log', type=str, default='',
                    help='path to the log file; `{auto}` can be used as part of the path to auto-generate a name, based on the timestamp and network configuration')
parser.add_argument('--print', action='store_true', default=False,
                    help='do not run training/prediction but only print model information, e.g., FLOPs and number of parameters of a model')
parser.add_argument('--profile', action='store_true', default=False,
                    help='run the profiler')
parser.add_argument('--backend', type=str, choices=['gloo', 'nccl', 'mpi'], default=None,
                    help='backend for distributed training')
parser.add_argument('--cross-validation', type=str, default=None,
                    help='enable k-fold cross validation; input format: `variable_name%%k`')
parser.add_argument('--disable-mps', action='store_true', default=False,
                    help='disable using mps device if it does not work for you')
parser.add_argument('--hidden-states-out', type=str, default="hidden_states_out.h5",
                    help='path to save hidden states as h5 file')
parser.add_argument('--hidden-states', action='store_true', default=False,
                    help='let ParT output hidden states with the logits')

_StoreTrueAction(option_strings=['--hidden-states'], dest='hidden_states', nargs=0, const=True, default=False, type=None, choices=None, help='let ParT output hidden states with the logits', metavar=None)

manually parse args

In [33]:
args = parser.parse_args([
    "--predict",
    "--data-test", "/Users/billyli/scope/JetClass/minimal/*.root",
    "--data-config", "/Users/billyli/scope/particle_transformer/data/JetClass/JetClass_full.yaml",
    "--data-fraction", "0.001",
    "--network-config", "/Users/billyli/scope/particle_transformer/networks/ParT_w_hidden_states.py",
    "--use-amp",
    "--model-prefix", "/Users/billyli/scope/particle_transformer/models/ParT_full.pt",
    "--batch-size", "2048",
    "--predict-output", "/Users/billyli/scope/particle_transformer/tmp/test_output.root",
    "--gpus", "",
    "--num-workers", "0",
    "--disable-mps",
    "--hidden-states"
])

print(args)

Namespace(backend=None, batch_size=2048, copy_inputs=False, cross_validation=None, data_config='/Users/billyli/scope/particle_transformer/data/JetClass/JetClass_full.yaml', data_fraction=0.001, data_test=['/Users/billyli/scope/JetClass/minimal/*.root'], data_train=[], data_val=[], demo=False, disable_mps=True, exclude_model_weights=None, export_onnx=None, extra_selection=None, extra_test_selection=None, fetch_by_files=False, fetch_step=0.01, file_fraction=1, freeze_model_weights=None, gpus='', hidden_states=True, hidden_states_out='hidden_states_out.h5', in_memory=False, io_test=False, load_epoch=None, load_model_weights=None, log='', lr_finder=None, lr_scheduler='flat+decay', model_prefix='/Users/billyli/scope/particle_transformer/models/ParT_full.pt', network_config='/Users/billyli/scope/particle_transformer/networks/ParT_w_hidden_states.py', network_option=[], no_remake_weights=False, num_epochs=20, num_workers=0, onnx_opset=15, optimizer='ranger', optimizer_option=[], predict=True,

## Dataloader

In [34]:
test_loaders, data_config = test_load(args)

## Load Model

In [35]:
if args.gpus:
    # distributed training
    if args.backend is not None:
        local_rank = args.local_rank
        torch.cuda.set_device(local_rank)
        gpus = [local_rank]
        dev = torch.device(local_rank)
        torch.distributed.init_process_group(backend=args.backend)
        _logger.info(f'Using distributed PyTorch with {args.backend} backend')
    else:
        gpus = [int(i) for i in args.gpus.split(',')]
        dev = torch.device(gpus[0])
else:
    gpus = None
    dev = torch.device('cpu')
    if not args.disable_mps:
        try:
            if torch.backends.mps.is_available():
                dev = torch.device('mps')
        except AttributeError:
            pass

In [36]:
model, model_info, loss_func = model_setup(args, data_config, device=dev)
model = model.to(dev)
model_path = args.model_prefix if args.model_prefix.endswith(
                '.pt') else args.model_prefix + '_best_epoch_state.pt'
_logger.info('Loading model %s for eval' % model_path)
model.load_state_dict(torch.load(model_path, map_location=dev))
if gpus is not None and len(gpus) > 1:
    model = torch.nn.DataParallel(model, device_ids=gpus)
model = model.to(dev)

/Users/billyli/miniforge_x86_new/envs/weaver/lib/python3.7/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


## Test run

In [37]:
for name, get_test_loader in test_loaders.items():
    test_loader = get_test_loader()

In [45]:
model.eval()
data_config = test_loader.dataset.config

label_counter = Counter()
total_loss = 0
num_batches = 0
total_correct = 0
entry_count = 0
count = 0
scores = []
labels = defaultdict(list)
labels_counts = []
observers = defaultdict(list)
hiddens = []
start_time = time.time()

In [46]:
with torch.no_grad():
     with tqdm(test_loader) as tq:
            for X, y, Z in tq:
                # X, y: torch.Tensor; Z: ak.Array
                inputs = [X[k].to(dev) for k in data_config.input_names]
                label = y[data_config.label_names[0]].long().to(dev)
                entry_count += label.shape[0]

                try:
                    mask = y[data_config.label_names[0] + '_mask'].bool().to(dev)
                except KeyError:
                    mask = None
                model_output = model(*inputs)

0it [00:00, ?it/s]

=== Restarting DataIter test_, seed=None ===


1it [00:33, 33.67s/it]


In [47]:
model_output

(tensor([[  1.1246,   6.0190,  -2.6362,  ...,  -4.2973,   0.8186,  -4.9007],
         [  1.9101,   9.9116,  -1.4260,  ...,  -6.6876,   2.2426,  -2.2734],
         [ -0.3478,   9.3876,   1.0444,  ...,  -2.9235,  -2.8806,  -4.1860],
         ...,
         [  0.6245,  -3.0741,  -0.0928,  ...,   0.9935,  -5.9286,  -7.9801],
         [  2.5972,   2.4498,  -2.8436,  ...,  -1.3412,  -3.6248,  -4.2935],
         [  0.0906,  -5.8252,  -2.1432,  ...,   2.8299,  -5.6116, -10.4245]]),
 [tensor([[[-2.5995e+00,  1.0335e+01,  1.1273e+00,  ...,  9.6274e-02,
             7.4875e+00, -1.5100e+01],
           [ 4.8566e+00, -3.1406e-01,  7.7696e-01,  ...,  3.0995e+00,
            -4.7420e+00,  5.5170e-01],
           [-8.3483e+00,  1.8832e+00,  1.0742e+00,  ...,  1.1236e+01,
            -4.3988e+00,  2.3959e+00],
           ...,
           [ 1.5444e+00,  5.6625e+00,  2.5174e+00,  ...,  6.1128e+00,
             2.5496e+00, -9.3715e+00],
           [-4.6162e+00,  9.7309e+00,  3.4943e+00,  ..., -5.2177e-01,
